In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kaggle-winning-solutions-methods/kaggle_winning_solutions_methods_detail.csv
/kaggle/input/kaggle-winning-solutions-methods/kaggle_winning_solutions_methods.csv
/kaggle/input/kaggle-winning-solutions-methods/method_category_mapping.csv
/kaggle/input/gemma/transformers/2b-it/2/model.safetensors.index.json
/kaggle/input/gemma/transformers/2b-it/2/gemma-2b-it.gguf
/kaggle/input/gemma/transformers/2b-it/2/config.json
/kaggle/input/gemma/transformers/2b-it/2/model-00001-of-00002.safetensors
/kaggle/input/gemma/transformers/2b-it/2/model-00002-of-00002.safetensors
/kaggle/input/gemma/transformers/2b-it/2/tokenizer.json
/kaggle/input/gemma/transformers/2b-it/2/tokenizer_config.json
/kaggle/input/gemma/transformers/2b-it/2/special_tokens_map.json
/kaggle/input/gemma/transformers/2b-it/2/.gitattributes
/kaggle/input/gemma/transformers/2b-it/2/tokenizer.model
/kaggle/input/gemma/transformers/2b-it/2/generation_config.json


In [2]:
!pip install -q -U huggingface_hub
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [3]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
access_token = UserSecretsClient().get_secret("Hugging Face")
login(token=access_token)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
%%time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Check what type of Device enabled (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
# Load the model
quantization_config = BitsAndBytesConfig(
                                        load_in_4bit=True,
                                        bnb_4bit_use_double_quant=True,
                                        bnb_4bit_quant_type="nf4",
                                        bnb_4bit_compute_dtype=torch.bfloat16,)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/2b-it/2")
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/2b-it/2", quantization_config=quantization_config, low_cpu_mem_usage=True)# Use the model

cuda:0


Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 10.2 s, sys: 5.66 s, total: 15.8 s
Wall time: 44.2 s


In [5]:
%%time
input_text = "What is the best thing about Kaggle?"
# Encode input text to PyTorch tensors
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, do_sample=True, max_new_tokens=100, temperature=0.5)

print(tokenizer.decode(outputs[0]))

2024-03-22 13:10:42.953510: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 13:10:42.953642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 13:10:43.091045: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<bos>What is the best thing about Kaggle?

**Answer:**

**The best thing about Kaggle is its vast and diverse dataset of labeled and unlabeled data, spanning across various domains and industries.** This allows users to explore and analyze problems from diverse perspectives, fostering creativity and problem-solving skills. Additionally, Kaggle offers several features that enhance the data exploration and analysis process, including:

* **Interactive data exploration tools:** Users can visualize data through interactive dashboards and explore data using various filtering and sorting options.
* **Community features
CPU times: user 12.6 s, sys: 1.06 s, total: 13.7 s
Wall time: 17.7 s


In [6]:
import pandas as pd
data = pd.read_csv('/kaggle/input/kaggle-winning-solutions-methods/kaggle_winning_solutions_methods.csv')
data.head(2)

,link,place,competition_name,prize,team,kind,metric,year,nm,writeup,num_tokens,methods,cleaned_methods
0,https://www.kaggle.com/c/asl-signs/discussion/...,2,Google - Isolated Sign Language Recognition,"$100,000","1,165",Research,PostProcessorKernelDesc,2023,406306,<h2>TLDR</h2>\n<p>We used an approach similar ...,2914,"['EfficientNet-B0', 'Data Augmentation', 'Norm...",Replace augmentation
1,https://www.kaggle.com/c/asl-signs/discussion/...,2,Google - Isolated Sign Language Recognition,"$100,000","1,165",Research,PostProcessorKernelDesc,2023,406306,<h2>TLDR</h2>\n<p>We used an approach similar ...,2914,"['EfficientNet-B0', 'Data Augmentation', 'Norm...",Finger tree rotate


In [7]:
data['writeup'][1]

'<h2>TLDR</h2>\n<p>We used an approach similar to audio spectrogram classification using the EfficientNet-B0 model, with numerous augmentations and transformer models such as BERT and DeBERTa as helper models. The final solution consists of one EfficientNet-B0 with an input size of 160x80, trained on a single fold from 8 randomly split folds, as well as DeBERTa and BERT trained on the full dataset. A single fold model using EfficientNet has a CV score of 0.898 and a leaderboard score of ~0.8.</p>\n<p>We used only competition data.</p>\n<h2>1. Data Preprocessing</h2>\n<h3>1.1 CNN Preprocessing</h3>\n<ul>\n<li>We extracted 18 lip points, 20 pose points (including arms, shoulders, eyebrows, and nose), and all hand points, resulting in a total of 80 points.</li>\n<li>During training, we applied various augmentations.</li>\n<li>We implemented standard normalization.</li>\n<li>Instead of dropping NaN values, we filled them with zeros after normalization.</li>\n<li>We interpolated the time ax

In [8]:
context = "Competition Name: " + data['competition_name'][1] + \
    ",\nYear: " + data['year'][1].astype(str) + \
    ",\nPlace: " + data['place'][1].astype(str) + \
    ",\nMethods Used: " + data['methods'][1] + \
    ",\nSolution: " + data['writeup'][1]


In [9]:
print(context)

Competition Name: Google - Isolated Sign Language Recognition,
Year: 2023,
Place: 2,
Methods Used: ['EfficientNet-B0', 'Data Augmentation', 'Normalization', 'Interpolation', 'BERT', 'DeBERTa', 'Mixup', 'Replace augmentation', 'Time and frequence masking', 'Random affine', 'Random interpolation', 'Flip pose', 'Finger tree rotate', 'Onecycle scheduler', 'Weighted CrossEntropyLoss', 'Hypercolumn', 'Ranger optimizer', 'Transformer', 'Knowledge distillation', 'Optuna', 'Label smoothing'],
Solution: <h2>TLDR</h2>
<p>We used an approach similar to audio spectrogram classification using the EfficientNet-B0 model, with numerous augmentations and transformer models such as BERT and DeBERTa as helper models. The final solution consists of one EfficientNet-B0 with an input size of 160x80, trained on a single fold from 8 randomly split folds, as well as DeBERTa and BERT trained on the full dataset. A single fold model using EfficientNet has a CV score of 0.898 and a leaderboard score of ~0.8.</p>
<

In [10]:
data['context'] = ("Competition Name: " + data['competition_name'] + \
  ",\nYear: " + data['year'].astype(str) + \
   ",\nPlace: " + data['place'].astype(str) + \
   ",\nMethods Used: " + data['methods'] + \
   ",\nSolution: " + data['writeup'])

context = data['context'][1]
print(context)

Competition Name: Google - Isolated Sign Language Recognition,
Year: 2023,
Place: 2,
Methods Used: ['EfficientNet-B0', 'Data Augmentation', 'Normalization', 'Interpolation', 'BERT', 'DeBERTa', 'Mixup', 'Replace augmentation', 'Time and frequence masking', 'Random affine', 'Random interpolation', 'Flip pose', 'Finger tree rotate', 'Onecycle scheduler', 'Weighted CrossEntropyLoss', 'Hypercolumn', 'Ranger optimizer', 'Transformer', 'Knowledge distillation', 'Optuna', 'Label smoothing'],
Solution: <h2>TLDR</h2>
<p>We used an approach similar to audio spectrogram classification using the EfficientNet-B0 model, with numerous augmentations and transformer models such as BERT and DeBERTa as helper models. The final solution consists of one EfficientNet-B0 with an input size of 160x80, trained on a single fold from 8 randomly split folds, as well as DeBERTa and BERT trained on the full dataset. A single fold model using EfficientNet has a CV score of 0.898 and a leaderboard score of ~0.8.</p>
<

In [11]:
def generate_summary():
    prompt_template = f"""Provide summary of following context in 500 words. 

    Provide only useful information: 
    
    Context: {context}"""


    messages = [
        {"role": "user", "content": prompt_template},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer.encode(prompt, add_special_tokens=True, return_tensors="pt").to('cuda')
    
    outputs = model.generate(input_ids, max_new_tokens=900)
    
    response = tokenizer.decode(outputs[0])
    return response

In [12]:
import re
def extract_content(text):
    # Find the index of '<start_of_turn>model'
    index = text.find('<start_of_turn>model')

    # Extract the content after '<start_of_turn>model'
    if index != -1:
        content_after_model = text[index + len('<start_of_turn>model'):].strip()
    else:
        return "Content not found after '<start_of_turn>model'"
    return content_after_model


In [13]:
summary = generate_summary()
final_summary = extract_content(summary)
print(final_summary)

Sure, here's a summary of the context in 500 words:

The context describes the development and training of an EfficientNet-B0 model for isolated sign language recognition. The model is trained on a single fold from 8 randomly split folds, using a combination of common and augmentation augmentations.

**Data Preprocessing:**

* 80 lip and 21 hand points are extracted from the image.
* Various augmentations and normalizations are applied to the extracted features.
* Motion features are included, including future and history motion.
* Finger tree rotation is used to generate features for the fingers.

**Training:**

* EfficientNet-B0 is trained on one fold with a random split.
* A onecycle scheduler with 0.1 warmup is used.
* Weighted CrossEntropyLoss is used to improve the model's performance.
* A hypercolumn is implemented for EfficientNet.
* BERT and DeBERTa are trained on the full dataset.

**Ensemble:**

* An ensemble of models is created by aggregating the outputs of the individual 

In [14]:
from IPython.display import Markdown as md
md(final_summary)

Sure, here's a summary of the context in 500 words:

The context describes the development and training of an EfficientNet-B0 model for isolated sign language recognition. The model is trained on a single fold from 8 randomly split folds, using a combination of common and augmentation augmentations.

**Data Preprocessing:**

* 80 lip and 21 hand points are extracted from the image.
* Various augmentations and normalizations are applied to the extracted features.
* Motion features are included, including future and history motion.
* Finger tree rotation is used to generate features for the fingers.

**Training:**

* EfficientNet-B0 is trained on one fold with a random split.
* A onecycle scheduler with 0.1 warmup is used.
* Weighted CrossEntropyLoss is used to improve the model's performance.
* A hypercolumn is implemented for EfficientNet.
* BERT and DeBERTa are trained on the full dataset.

**Ensemble:**

* An ensemble of models is created by aggregating the outputs of the individual models.
* The ensemble weights are calculated based on the local fold 0 score.
* The final ensemble includes EfficientNet-B0, BERT, and DeBERTa.

**Results:**

* EfficientNet-B0 achieves a leaderboard score of approximately 0.8.
* Transformers improve the score to 0.81.
* The final ensemble includes EfficientNet-B0, BERT, and DeBERTa.

**Key Takeaways:**

* The model uses a combination of common and augmentation augmentations.
* The ensemble weights are calculated based on the local fold 0 score.
* EfficientNet-B0 is a highly effective model for isolated sign language recognition.

**Additional Notes:**

* The model is trained on a single fold, which may not be representative of the entire dataset.
* The model is rewritten in Keras to improve performance.
* The running time of the DepthwiseConv2D operation is significantly higher than the other operations.<eos>